In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# load local version of gallifrey, before pip installed version, for debugging
import pathlib
import sys
import os

sys.path.append(str(pathlib.Path(os.getcwd()).parent.joinpath("src")))

In [3]:
import yt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from gallifrey.model import Model
from gallifrey.analysis import count_planets
from gallifrey.utilities.math import calculate_pca
from gallifrey.utilities.dataframe import rename_labels
from gallifrey.visualization.visualization_utils import (
    set_plot_defaults,
    get_palette,
    FigureProcessor,
    adjust_legend,
    ridgeplot,
)

## DATA SETUP

In [4]:
snapshot = 127
resolution = 8192
sim_id = "37_11"
num_embryos = 50
host_star_masses = 1
planet_categories = [
    "Earth",
    "Super-Earth",
    "Neptunian",
    "Sub-Giant",
    "Giant",
]

In [5]:
model = Model(
    snapshot=snapshot,
    resolution=resolution,
    sim_id=sim_id,
)
ds = model.ds
mw = model.halo


      DETECTED LOCAL MACHINE: Test snapshot loaded.


      CONFIG: Setting resolution to 4096.


      CONFIG: Setting sim_id to 09_18.

yt : [INFO     ] 2023-10-30 18:13:15,645 Calculating time from 1.000e+00 to be 4.318e+17 seconds
yt : [INFO     ] 2023-10-30 18:13:15,755 Parameters: current_time              = 4.318436660023809e+17 s
yt : [INFO     ] 2023-10-30 18:13:15,756 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2023-10-30 18:13:15,756 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2023-10-30 18:13:15,757 Parameters: domain_right_edge         = [100. 100. 100.]
yt : [INFO     ] 2023-10-30 18:13:15,758 Parameters: cosmological_simulation   = 1
yt : [INFO     ] 2023-10-30 18:13:15,758 Parameters: current_redshift          = 4.440892098500626e-16
yt : [INFO     ] 2023-10-30 18:13:15,758 Parameters: omega_lambda              = 0.682
yt : [INFO     ] 2023-10-30 18:13:15,759 Parameters: omega_matter              = 0.318
yt : [INFO     ] 202

## PLOT SETUP

In [6]:
set_plot_defaults()

figure_subdirectory = f"MilkyWayAnalogue/{model.resolution}/{model.sim_id}/{model.snapshot}/Planet_numbers"
save = True

## CREATE DATA SOURCE

In [7]:
radius = 0.2 * mw.virial_radius()

## ABSOLUTE NUMBER PER PLANET TYPE

In [8]:
def aggregate_planet_counts_for_embryos(host_star_mass, embryo_list):
    counts_by_embryos = {
        num_embryos: count_planets(
            model=model,
            data_creator=lambda: mw.sphere(radius=radius),
            planet_categories=planet_categories,
            model_config={
                "ngpps_star_masses": host_star_mass,
                "ngpps_num_embryos": num_embryos,
            },
            components="stars",
        )
        for num_embryos in embryo_list
    }
    counts_dataframe = pd.concat(counts_by_embryos).reset_index(level=1, drop=True)
    return counts_dataframe.rename_axis(r"$N_\mathrm{Embryos}$")

In [9]:
embryo_list = (10, 20, 50, 100)

planet_counts = aggregate_planet_counts_for_embryos(
    host_star_mass=1, embryo_list=embryo_list
)


      CONFIG: Setting ngpps_num_embryos to 10.


      TIMER: Block 'Adding Planets...' took 0.86 seconds.


      CONFIG: Setting ngpps_num_embryos to 20.


      TIMER: Block 'Adding Planets...' took 1.23 seconds.


      CONFIG: Setting ngpps_num_embryos to 50.


      TIMER: Block 'Adding Planets...' took 1.6 seconds.


      CONFIG: Setting ngpps_num_embryos to 100.


      TIMER: Block 'Adding Planets...' took 1.95 seconds.



In [13]:
print("ALSO DO FOR OTHER MASSES, DISCUSS THE OBSERVATIONAL ARGUMENTS (EARTHS MORE COMMON THAN SUPER-EARTHS -> NOT LIKE IN OBSERVATIONS, PLANETS AROUND DWARFS RARER -> NOT IN OBSERVATIONS)")
print("COMPARE AND RESCALE TO OBSERVATIONAL VALUES (PLATO PAPER MATUSZEWSKI2023")
planet_counts

ALSO DO FOR OTHER MASSES, DISCUSS THE OBSERVATIONAL ARGUMENTS (EARTHS MORE COMMON THAN SUPER-EARTHS -> NOT LIKE IN OBSERVATIONS, PLANETS AROUND DWARFS RARER -> NOT IN OBSERVATIONS)
COMPARE AND RESCALE TO OBSERVATIONAL VALUES (PLATO PAPER MATUSZEWSKI2023


,Earth,Super-Earth,Neptunian,Sub-Giant,Giant
$N_\mathrm{Embryos}$,,,,,
10,3.544949e+09,4.201226e+09,1.332038e+09,3.457049e+08,7.928479e+08
20,6.097034e+09,5.837326e+09,1.455560e+09,3.714971e+08,7.945205e+08
50,1.104743e+10,1.018676e+10,1.342733e+09,3.053752e+08,6.806437e+08
100,1.292790e+10,1.226568e+10,1.468761e+09,3.133132e+08,7.216519e+08
